In [602]:
import numpy as np
import matplotlib.pyplot as plt
import pandas
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout
from tensorflow.keras.layers import Layer,Input,InputSpec
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from tensorflow.keras.layers import Conv2D,Flatten,ConvLSTM2D,Conv1D
from tensorflow.keras import backend as K

from tensorflow.python.ops import math_ops
from tensorflow.keras.backend import argmax
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from reformat import ReformatLayer

from tensorflow.keras.utils import to_categorical


In [639]:
class ReformatLayer(Layer):
    def __init__(self, n_window,   **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(ReformatLayer, self).__init__(**kwargs)
        self.n_window = n_window
        #self.initial_weights = weights
        self.input_spec = InputSpec(ndim=2)

    def _MinMax(self,aInp):
        xf = K.cast(aInp, 'float32') 
        x = math_ops.div_no_nan(
            math_ops.subtract( xf,  tf.reduce_min(xf)            ), 
            math_ops.subtract(tf.reduce_max(xf),    tf.reduce_min(xf)) 
            )
        return x
    def _create_data(self,ainp,aWindow):
        xx=ainp
        for i in range(aWindow-1):
            x=tf.roll(ainp,-i-1,0)
            xx=tf.concat([xx,x],1)
        #x2=tf.roll(ainp,[-1,-1],[-1,2])
        #xx2=tf.concat([xx,x2],1)
        #x3=tf.roll(ainp,[-1,-1],[-1,3])
        #xx3=tf.concat([xx2,x3],1)
        print('x2=',xx)
        return xx    

    def create_dataset(self, dataset, look_back=1):
        from sklearn.preprocessing import MinMaxScaler
        print('dataset ',dataset) 
        print('datshape',dataset.get_shape()[0])
        ln=dataset.get_shape()[0]
        #scaler = MinMaxScaler(feature_range=(0, 0.999))
        #YM = scaler.fit_transform(dataset)
        dataX=[]
        dataY =  []
        if ln is not None:
            for i in range(ln-look_back):
                a = dataset[i:(i+look_back)]
                dataY.append(dataset[i + look_back]) 
                dataX.append(a)
        #dataXX= numpy.reshape(dataX, (dataX.shape[0], dataX.shape[1],1))
    
        return np.array(dataX), np.array(dataY) 

    def build(self, input_shape):
        print('build', input_shape)
        #input_dim = input_shape[1]
        self.input_spec = InputSpec(dtype=K.floatx(), shape=(None,1 ))
        self.total = tf.Variable(initial_value=tf.ones((input_shape[1],)),
                             trainable=False)

        #self.input_spec = [InputSpec(shape=input_shape)]
        # assert len(input_shape) == 1
        #input_dim = input_shape[1]
        #self.input_spec = InputSpec(dtype=tf.int32, shape=(None, input_dim))
        #self.clusters = self.add_weight('clusters',(self.n_clusters, input_dim), initializer='glorot_uniform')
        self.built = True

    def call(self, inputs, **kwargs):
        print('call',inputs)
        xx0=self._MinMax(inputs)
        print('minmax=',xx0)
        #x,y=self.create_dataset(inputs,self.n_window)
        x=self._create_data(xx0,self.n_window)
        print('x.shape',x.shape)
        x1= np.reshape(x, (x.shape[0], 1,x.shape[1]))
        print('xq=',x1.shape)
        return x1

    def compute_output_shape(self, input_shape):
        print('output',input_shape)
        assert input_shape and len(input_shape) == 2
        return input_shape[0], self.n_window

    def get_config(self):
        config = {'n_window': self.n_window}
        base_config = super(ReformatLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
     

In [640]:
x = tf.ones((1),dtype=tf.int32)
#x=K.reshape(x,(None,1))
x

<tf.Tensor: id=11673, shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>

In [641]:
mm=np.array([1,2,3])
mmm=mm.astype(K.floatx())
mmm

array([1., 2., 3.], dtype=float32)

In [642]:
y=[[1],[2],[3],[5],[6],[7],[8]]
#y=[[]]

#xx=tf.concat([[]],0)
xx=np.array(y[0:len(y)-3])
#x1=np.array(y[1:len(y)-2])
#xx=tf.concat([y,y],1)
for i in range(3):
    x1=y[i+1:len(y)-2+i]
    
    xx=tf.concat([xx,x1],1)
    
xx

<tf.Tensor: id=11684, shape=(4, 4), dtype=int64, numpy=
array([[1, 2, 3, 5],
       [2, 3, 5, 6],
       [3, 5, 6, 7],
       [5, 6, 7, 8]])>

In [643]:
#xx = tf.concat([y,[[12],[13],[14],[15],[16],[17]]],1)
xx= tf.concat([y,y],1)
#xx = tf.concat([xx,[[12],[13],[14],[15],[16],[17]]],1)
#xx = tf.concat([xx,[[12],[13],[14],[15],[16],[17]]],1)
xx
#xx2=K.reshape(xx,(-1,5))
#xx2.shape

<tf.Tensor: id=11689, shape=(7, 2), dtype=int32, numpy=
array([[1, 1],
       [2, 2],
       [3, 3],
       [5, 5],
       [6, 6],
       [7, 7],
       [8, 8]], dtype=int32)>

In [644]:
x = tf.ones((10),dtype=tf.int32)
y=[3,4,5,6,7]
z=tf.concat([y,x,y],0)
z

<tf.Tensor: id=11697, shape=(20,), dtype=int32, numpy=
array([3, 4, 5, 6, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 4, 5, 6, 7],
      dtype=int32)>

In [645]:
zt2=K.reshape(z,(-1,1))
zt2

<tf.Tensor: id=11700, shape=(20, 1), dtype=int32, numpy=
array([[3],
       [4],
       [5],
       [6],
       [7],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [3],
       [4],
       [5],
       [6],
       [7]], dtype=int32)>

In [646]:
#x=[1,1]
#tf.backend.eval(x)
wi=Input([1,2,3])
z2=tf.constant([])
z1=tf.constant([2])
#z2.append(z1)
#z=tf.Variable([1])
#z=tf.Tensor([1,2,3],dtype=tf.int32)
z

<tf.Tensor: id=11697, shape=(20,), dtype=int32, numpy=
array([3, 4, 5, 6, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 4, 5, 6, 7],
      dtype=int32)>

In [647]:
linear_layer = ReformatLayer( 5)
y = linear_layer(zt2)
print(y)

build (20, 1)
call tf.Tensor(
[[3]
 [4]
 [5]
 [6]
 [7]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [4]
 [5]
 [6]
 [7]], shape=(20, 1), dtype=int32)
minmax= tf.Tensor(
[[0.33333334]
 [0.5       ]
 [0.6666667 ]
 [0.8333333 ]
 [1.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.33333334]
 [0.5       ]
 [0.6666667 ]
 [0.8333333 ]
 [1.        ]], shape=(20, 1), dtype=float32)
x2= tf.Tensor(
[[0.33333334 0.5        0.6666667  0.8333333  1.        ]
 [0.5        0.6666667  0.8333333  1.         0.        ]
 [0.6666667  0.8333333  1.         0.         0.        ]
 [0.8333333  1.         0.         0.         0.        ]
 [1.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.    

In [651]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataY.append(dataset[i + look_back, 0]) 
        dataX.append(a)
    return numpy.array(dataX), numpy.array(dataY)

In [ ]:
zzz=

array([[3, 4, 5, 6, 7],
       [4, 5, 6, 7, 1],
       [5, 6, 7, 1, 1],
       [6, 7, 1, 1, 1],
       [7, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 3],
       [1, 1, 1, 3, 4],
       [1, 1, 3, 4, 5]], dtype=int32)

In [657]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 0.99999))
YM = scaler.fit_transform(zt2)
#YM=Y.loc[:,:]
YM

#numpy.bincount(Y.iloc[:,0])

array([[0.33333 ],
       [0.499995],
       [0.66666 ],
       [0.833325],
       [0.99999 ],
       [0.      ],
       [0.      ],
       [0.      ],
       [0.      ],
       [0.      ],
       [0.      ],
       [0.      ],
       [0.      ],
       [0.      ],
       [0.      ],
       [0.33333 ],
       [0.499995],
       [0.66666 ],
       [0.833325],
       [0.99999 ]])

In [658]:
zz1,zz2=create_dataset(YM,5)
zz1

array([[0.33333 , 0.499995, 0.66666 , 0.833325, 0.99999 ],
       [0.499995, 0.66666 , 0.833325, 0.99999 , 0.      ],
       [0.66666 , 0.833325, 0.99999 , 0.      , 0.      ],
       [0.833325, 0.99999 , 0.      , 0.      , 0.      ],
       [0.99999 , 0.      , 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , 0.      , 0.33333 ],
       [0.      , 0.      , 0.      , 0.33333 , 0.499995],
       [0.      , 0.      , 0.33333 , 0.499995, 0.66666 ]])

In [659]:
zz1.shape

(14, 5)

In [648]:
class ComputeSum(Layer):

    def __init__(self):
        super(ComputeSum, self).__init__()
        #self.total = tf.Variable(initial_value=tf.zeros((input_dim,)),
        #                     trainable=False)

    def build(self, input_shape):
        print('build', input_shape)
        print('bmm',input_shape[-1])
        #init = tf.constant([1,2,3,4])
        self.w = tf.Variable(initial_value=
                             tf.ones((input_shape[-1],20)),
                             trainable=False)
        #self.w = self.add_weight(shape=(input_shape[-1], 3),
        #                     initializer='random_normal',
        #                     trainable=False)
        #self.w=np.array([1.,2.,3.]
    def call(self, inputs):
        print('call',inputs)
        print('w',self.w)
        return tf.matmul(inputs, self.w) 
        #self.total.assign_add(tf.reduce_sum(inputs, axis=0))
        #return self.total

In [649]:
x = tf.concat([[1.,2.,3,4,5,6]],0)
x=tf.reshape(x,[-1,1])
x1 = tf.concat([[2.,3,4,5,6,7]],0)
x1=tf.roll(x1,-2,0)
x1=tf.reshape(x1,[-1,1])
print('x1',x1)
#xn=tf.stack([x,x1],1)
xn=tf.concat([x,x1],1)
xn2=tf.concat([xn,x],1)
#x=Input(numpy.array([[1.,1.,2.][1,4,6]]))
#x=tf.reshape(x,(-1,1))
print('xshape',xn.shape)
xn3,xn4=tf.split(xn,[0],0)
#y=tf.roll(xn,2,0)
#y=tf.slice(x,[0,3],[-1 ,-1])
print(xn3)

x1 tf.Tensor(
[[4.]
 [5.]
 [6.]
 [7.]
 [2.]
 [3.]], shape=(6, 1), dtype=float32)
xshape (6, 2)


InvalidArgumentError: If there is only one output, it must have the same size as the input. Input size: 6 output size: 0 [Op:SplitV] name: split

In [474]:
x1=[1.,2,3]
x2=[3.,4,5]
z=tf.stack([x1,x2],1)
print(z)
z2=z[2:4]
z

tf.Tensor(
[[1. 3.]
 [2. 4.]
 [3. 5.]], shape=(3, 2), dtype=float32)


<tf.Tensor: id=7992, shape=(3, 2), dtype=float32, numpy=
array([[1., 3.],
       [2., 4.],
       [3., 5.]], dtype=float32)>

In [384]:
my_sum = ComputeSum()
y = my_sum(x)
#print(y.numpy())
y
#y = my_sum(x)
#print(y.numpy())

build (6, 1)
bmm 1
call tf.Tensor(
[[1.]
 [2.]
 [3.]
 [4.]
 [5.]
 [6.]], shape=(6, 1), dtype=float32)
w <tf.Variable 'compute_sum_36/Variable:0' shape=(1, 20) dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.]], dtype=float32)>


<tf.Tensor: id=6381, shape=(6, 20), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.],
       [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2.],
       [3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3.],
       [4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
        4., 4., 4., 4.],
       [5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
        5., 5., 5., 5.],
       [6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
        6., 6., 6., 6.]], dtype=float32)>